In [5]:
import tensorflow as ts
from keras.preprocessing import image
import keras
import sys

In [6]:

image.ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None)

In [13]:
import numpy as np
import np_utils
from keras.models import Model
from keras.applications.vgg16 import VGG16

In [16]:

model = VGG16(weights='imagenet', include_top=False)

In [18]:

path = 'shoml_data/athletic_flatten'
out_path = 'shoml_data/athletic_augmented_test'

train_datagen = image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        path,
        class_mode='binary')

#model.fit_generator(
   #      train_generator,
   #     steps_per_epoch=2000,
   #     epochs=50)


Found 0 images belonging to 0 classes.


In [20]:
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import rotate
import scipy.misc